In [ ]:
# imports
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
# set random generator
generator = np.random.RandomState(42)

# settings of graphics
width = 17
height = 5
matplotlib.rcParams['figure.figsize'] = [width, height]
matplotlib.rcParams['lines.markersize'] = 0.5
matplotlib.rcParams['scatter.edgecolors'] = "black"

# autoregression generator
# p               memory parameter of autoregression, 0 <= p <= 1
# base_value      first value, from which the generation is initialized
# noise_maker     function of time which produces noise
# to_add          number of elements in result
def ar(p, base_value, noise_maker, to_add = 10000):
    new_vals = np.array(arr)
    new_vals = np.append(new_vals, [0] * to_add)
    pos = 1
    for i in range(to_add):
        noise = noise_maker(i)
        new_vals[pos + i] = p * new_vals[pos + i - 1] + noise
    return new_vals

# fourier transform
# returns array of amplitudes correcponding to different periods
# a        input array
# draw     to draw or not to draw the graphics
# def mfft(a, draw=False):
#     plt.rcParams['axes.grid'] = True
#     plt.tight_layout()
#     A = np.fft.rfft(a)
#     N = len(a)
#     if (draw):
#         plt.rcParams['axes.grid'] = True
#         fig, ax = plt.subplots(2, figsize=(6,4), dpi=150)
#         plt.tight_layout()
#         n = np.arange(len(a))
        
#         ax[0].plot(n, a, '.-')
#         ax[0].set_title('$a[n]$')
#         ax[1].set_xlim(0, 170)
#         ax[1].set_title('$A$')
#         ax[1].plot(n[0 : (N // 2 + 1)], np.abs(A), '-')
        
#     return np.abs(A / N)


# yet another fourier transform
# returns normalized amplitudes (scaled by the number of points)
# arr    input array
# draw   to draw or not to draw graphics of base data and amplitudes
# name   name of the feature
# x1     lower limit of periods to be displayed
# x2     upper limit of periods to be displayed
# y      upper limit of amplitudes to be displayed
def mfft2(arr, draw=False, name='a[n]', x1=0, x2=-1, y=0):
    if x2 == -1:
        x2 = len(arr) // 2 + 1
    plt.rcParams['axes.grid'] = True
    plt.tight_layout()
    A = np.fft.rfft((arr - np.mean(arr)) / len(arr))
    n = np.arange(len(arr))
    n1 = len(arr) / n[1:]
    if (draw):
        plt.rcParams['axes.grid'] = True
        fig, ax = plt.subplots(2, figsize=(6,4), dpi=150)
        plt.tight_layout()
        plt.subplots_adjust(hspace = 0.5)
        ax[0].plot(n, arr, '.-')
        ax[0].set_title(name)
        ax[0].set_ylim(-100, 100)
        ax[1].set_title('$A$')
        ax[1].set_xlim(x1, x2)
        ax[1].plot(n1[0 : (len(arr) // 2 + 1)], np.abs(A), '-')
        ax[1].set_xlabel('Период (в днях)')
        ax[1].set_ylabel('Амплитуда')
        if y != 0:
            ax[1].set_ylim(0, y)
        plt.show()
        
    return zip(n1[0 : (len(arr) // 2 + 1)], np.abs(A))

In [ ]:
# draws graphic of feature which is presented in table from omniweb and its mfft2 result
# name    name of the feature
# xlim    upper limit of periods
# ylim    upper limit of amplitudes
def draw_feature(name, xlim=100, ylim=0):
    if ylim == 0:
        mfft2(data[name], True, name, x1=0, x2=xlim)
    else:
        mfft2(data[name], True, name, x1=0, x2=xlim, y=ylim)

# draws some array and its mfft2
# name         name of the feature
# data_arr     input array for processing
# xlim         upper limit iot periods
# ylim         upper limit of periods
def draw_arr_feature(name, data_arr, xlim=100, ylim=0):
    if ylim == 0:
        mfft2(data_arr, True, name, x1=0, x2=xlim)
    else:
        mfft2(data_arr, True, name, x1=0, x2=xlim, y=ylim)

In [ ]:
# checks if point x is in d-neighbourhood of center
def in_delta(x, center, d):
    return np.abs(x - center) < d

In [ ]:
# here is the variation of <<ar>> method,
# but the noise added in delta-neighbourhood of specific points differs
# from the noise in the rest of the area
# delta      the radius of sinusoidal noise area
# T          sinusoidal noise will have this period and cyclic frequency 2 * pi / T
# p          memory parametr in autoregression
# A_sin      amplitude of sinusoidal noise
def imitate_Dst(delta=45, T=27, p=1, A_sin=1):
    spring = 79
    autumn = 266
    freq = 2 * np.pi / T
    noiser_basic = lambda : generator.normal(0, 1)
    noiser_sin = lambda i : A_sin * generator.normal(np.sin(freq * i), 1)
    start_value = -20
    arr = [start_value]
    for i in range(N):
        new_val = p * arr[-1]
        day = i % 365
        if in_delta(day, spring, delta) or in_delta(day, autumn, delta):
            # add noise_sin
            new_val += noiser_sin(day)
        else:
            # add noise_basic
            new_val += noiser_basic()
        arr.append(new_val)
    
    return list(map(lambda t : t * 1, arr))


In [ ]:
from sklearn.metrics import mean_absolute_error

# mp    current value of p
# left
# right frame where we count the amplitude
def get_amplitude_of_p(mp, left, right):
    generated_data = imitate_Dst(p=mp, A_sin=1)
    generated_data = list(map(lambda x : x, generated_data))
    fft_res = list(mfft2(generated_data, False, ""))
    filtered = filter(lambda x : left <= x[0] <= right, fft_res)
    return(max(map(lambda x : x[1], filtered)))

def get_semiannual_amp(mp):
    return get_amplitude_of_p(mp, 175, 190)

def get_27day_amp(mp):
    return get_amplitude_of_p(mp, 24, 30)

def draw_semiannual_amplitude(p_from=0.5, p_to=1, n=100):
    plt.figure(figsize=(13, 5))
    plt.xlabel("value of p", fontsize=17)
    plt.ylabel("amplitude of semi-annual period", fontsize=17)
    ps = np.linspace(p_from, p_to, n)
    res = list(map(lambda px : get_semiannual_amp(px), ps))
    plt.title("Dependency of A_semiannual from p", fontsize=19)
    plt.plot(ps, res)
    
def draw_27day_amplitude(p_from=0.5, p_to=1, n=100):
    plt.figure(figsize=(13, 5))
    plt.xlabel("value of p", fontsize=17)
    plt.ylabel("amplitude of 27-day period", fontsize=17)
    ps = np.linspace(p_from, p_to, n)
    res = list(map(lambda px : get_27day_amp(px), ps))
    approx_polynom = np.polyfit(ps, res, 1)
    approx = np.polyval(approx_polynom, ps)
    
    plt.title("Dependency of A_27day from p", fontsize=19)
    plt.plot(ps, res)
    plt.plot(ps, approx)
    print("polynomial : ", approx_polynom)
    
    mape = mean_absolute_error(res, approx) / sum(list(map(abs, res)))
    print("mean average error : ", mape)
    
def draw_amplitude_relation(p_from=0.5, p_to=1, n=100):
    plt.figure(figsize=(13, 5))
    plt.xlabel("value of p", fontsize=17)
    plt.ylabel("relation A_semi/A_27 ", fontsize=17)
    ps = np.linspace(p_from, p_to, n)
    res = list(map(lambda x : get_semiannual_amp(x) / max(0.01, get_27day_amp(x)), ps))
    plt.title("Dependency of amplitude relationship from p", fontsize=19)
    plt.plot(ps, res)